In [1]:
import pandas as pd
import numpy as np

In [2]:
title_akas = pd.read_csv('title_akas.tsv', sep = '\t',  low_memory = False)
title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [3]:
title_rating = pd.read_csv('title_ratings.tsv', sep = '\t',  low_memory = False)
title_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1684
1,tt0000002,6.0,207
2,tt0000003,6.5,1421
3,tt0000004,6.1,121
4,tt0000005,6.1,2221


In [4]:
title_basics = pd.read_csv('title_basics.tsv', sep = '\t',  low_memory = False)
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
name_basics = pd.read_csv('name_basics.tsv', sep = '\t',  low_memory = False)
name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0071877,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0056404,tt0057345,tt0049189,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0077975,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0050976,tt0060827,tt0069467"


In [6]:
title_rating = pd.read_csv('title_ratings.tsv', sep = '\t',  low_memory = False)
title_rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1684
1,tt0000002,6.0,207
2,tt0000003,6.5,1421
3,tt0000004,6.1,121
4,tt0000005,6.1,2221


In [7]:
title_crew = pd.read_csv('title_crew.tsv', sep = '\t',  low_memory = False)
title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [8]:
movie = title_basics[title_basics['titleType'] == 'movie'].copy()
movie.drop(['originalTitle', 'isAdult', 'endYear', 'runtimeMinutes'], axis = 1, inplace = True)

title_rating['averageRating'] = title_rating.averageRating.astype(object).copy()
title_rating['numVotes'] = title_rating.numVotes.astype(object).copy()

movie = movie.join(title_rating.set_index('tconst'), on = ['tconst'])
movie = movie[movie['numVotes'] > 10000].copy()
movie = movie[movie['averageRating'] > 3.0].copy()
movie = movie[movie['startYear'].str.isdigit()].copy()

movie['startYear'] = movie['startYear'].astype(int)

movie = movie[movie['startYear'] > 2000].copy()
movie = movie[movie['genres'].str.find(r'\N') == -1].copy()




In [9]:
movie

,tconst,titleType,primaryTitle,startYear,genres,averageRating,numVotes
34810,tt0035423,movie,Kate & Leopold,2001,"Comedy,Fantasy,Romance",6.4,79136
117889,tt0120667,movie,Fantastic Four,2005,"Action,Adventure,Fantasy",5.7,312012
117899,tt0120679,movie,Frida,2002,"Biography,Drama,Romance",7.4,81605
117901,tt0120681,movie,From Hell,2001,"Horror,Mystery,Thriller",6.8,147161
117944,tt0120737,movie,The Lord of the Rings: The Fellowship of the Ring,2001,"Action,Adventure,Drama",8.8,1666847
...,...,...,...,...,...,...,...
7597654,tt9845564,movie,Below Zero,2021,"Action,Adventure,Crime",6.2,10656
7604874,tt9860728,movie,Falling Inn Love,2019,"Comedy,Romance",5.6,15927
7607422,tt9866072,movie,Holidate,2020,"Comedy,Romance",6.1,45176
7622955,tt9898858,movie,Coffee & Kareem,2020,"Action,Comedy,Crime",5.1,11765


In [10]:
for i in range(len(movie)):
    if movie.iloc[i]['genres'].find(',') != -1:
        genre_lst = []
        genre_lst = movie.iloc[i]['genres'].split(',')
        new = {'tconst' : [], 'titleType' : [], 'primaryTitle' : [], 'startYear' : [], 
               'genres' : [], 'averageRating' : [], 'numVotes' : []}
        for j in range(len(genre_lst)):
            new['tconst'].append(movie['tconst'].iloc[i])
            new['titleType'].append('movie')
            new['primaryTitle'].append(movie['primaryTitle'].iloc[i])
            new['startYear'].append(movie['startYear'].iloc[i])
            new['genres'].append(genre_lst[j])
            new['averageRating'].append(movie['averageRating'].iloc[i])
            new['numVotes'].append(movie['numVotes'].iloc[i])
        new = pd.DataFrame(new)
        movie = pd.concat([movie, new])
        
movie = movie[movie['genres'].str.find(',') == -1]
movie

,tconst,titleType,primaryTitle,startYear,genres,averageRating,numVotes
118014,tt0120824,movie,The Shipping News,2001,Drama,6.7,31890
188972,tt0196229,movie,Zoolander,2001,Comedy,6.5,246778
200799,tt0208990,movie,The Son's Room,2001,Drama,7.3,17638
200879,tt0209077,movie,Ken Park,2002,Drama,5.9,28789
230392,tt0240200,movie,Water,2005,Drama,7.7,14330
...,...,...,...,...,...,...,...
0,tt9898858,movie,Coffee & Kareem,2020,Action,5.1,11765
1,tt9898858,movie,Coffee & Kareem,2020,Comedy,5.1,11765
2,tt9898858,movie,Coffee & Kareem,2020,Crime,5.1,11765
0,tt9900782,movie,Kaithi,2019,Action,8.5,12492


In [11]:
rating = movie[['genres','averageRating']].copy()
rating['averageRating'] = rating.averageRating.astype(int).copy()
rating = rating.groupby('genres').mean()
rating.to_csv('rating.csv')

In [12]:
popular = movie[['genres','numVotes']].copy()
popular['numVotes'] = popular.numVotes.astype(int).copy()
popular = popular.groupby('genres').mean()
popular.to_csv('popular')

In [13]:
trend = movie[['startYear','genres','numVotes']].copy()
trend['numVotes'] = trend.numVotes.astype(int).copy()
trend = trend.groupby(['startYear','genres']).mean()
trend.reset_index(inplace = True)

In [14]:
tmp = trend[trend['startYear'] <= 2005].copy()
tmp.drop('startYear', inplace =  True, axis = 1)
trend_05 = tmp.groupby('genres').mean()
trend_05.to_csv('trend0-5')

In [15]:
tmp = trend[(trend['startYear'] <= 2010) & (trend['startYear'] > 2005)].copy()
tmp.drop('startYear', inplace =  True, axis = 1)
trend_510 = tmp.groupby('genres').mean()
trend_510.to_csv('trend5-10')

In [16]:
tmp = trend[(trend['startYear'] <= 2015) & (trend['startYear'] > 2010)].copy()
tmp.drop('startYear', inplace =  True, axis = 1)
trend_1015 = tmp.groupby('genres').mean()
trend_1015.to_csv('trend10-15')

In [17]:
tmp = trend[(trend['startYear'] <= 2020) & (trend['startYear'] > 2015)].copy()
tmp.drop('startYear', inplace =  True, axis = 1)
trend_1520 = tmp.groupby('genres').mean()
trend_1520.to_csv('trend15-20')